 El desafío consta de 3 ejercicios independientes que van desde análisis exploratorio, machine learning o el diseño de una solución de data science.

 ¿Qué evaluamos?

 El desafío busca evaluar distintos aspectos como:

- Capacidad analitica y exploración de datos
- Visualización de resultados
- Conocimientos de técnicas de generación de features y modelado
- Análisis de performance
- Buenas prácticas de desarrollo
- Diseño e implementación de Machine learning en producción

 Algunas reglas y recomendaciones:
 1. La mayoría de los ejercicios se piden resolver en Jupyter notebooks y te recomendamos subirlas a un repositorio de GitHub público para compartir los resultados.
 2. No dejes de hacernos preguntas sobre cualquier duda con los enunciados. El desafío se analiza de acuerdo al seniority del postulante y teniendo en cuenta también las necesidades particulares de la posición.

1. Explorar las ofertas relámpago, ¿qué insights puedes generar?

- __Descripción__

 En conjunto con el desafío te compartimos un archivo llamado "ofertas_relampago.csv" el cual posee información de los resultados de ofertas del tipo relampago para un periodo de tiempo y un país determinado.

 Estas ofertas en mercadolibre se pueden ver de la siguiente manera:

 Es decir, son ofertas que tienen una duración definida de algunas horas y un porcentaje de unidades (stock) comprometidas.

 El objetivo de este desafío es hacer un EDA sobre estos datos buscando insights sobre este tipo de ofertas.

 Las columnas del dataset son autoexplicativas pero puedes preguntarnos cualquier duda.

- __Entregable__

 El entregable de este desafío es una Jupyter notebook con el EDA.

2. Similitud entre productos:

- __Descripción__

 Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.

 Para este desafío tenemos un dataset "items_titles.csv" que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil

- __Entregable__

 El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset "item_titles_test.csv" generando como output un listado de la forma donde ordenando por score de similitud podamos encontrar los pares de productos más
 similares en nuestro dataset de test.

3. Previsión de falla

- __Descripción__

 Los galpones de Full de mercado libre cuentan con una flota de dispositivos que transmiten diariamente telemetría agregada en varios atributos.

 Las técnicas de mantenimiento predictivo están diseñadas para ayudar a determinar la condición del equipo de mantenimiento en servicio para predecir cuándo se debe realizar el mantenimiento. Este enfoque promete ahorros de costos sobre el mantenimiento preventivo
 de rutina o basado en el tiempo porque las tareas se realizan solo cuando están justificadas.

- __Entregable__

 Tiene la tarea de generar una Jupyter notebook con un modelo predictivo para predecir la probabilidad de falla del dispositivo con el objetivo de bajar los costos del proceso. Como una referencia, una falla de un dispositivo tiene un costo de 1 mientras el costo de un mantenimiento es 0,5. El archivo "full_devices.csv" tiene los valores diários para los 9 atributos de los dispositivos y la columna que está tratando de predecir se llama 'failure' con el valor binario 0 para no fallar y 1 para fallar

In [1]:
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('rslp')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import RSLPStemmer

import spacy
!python -m spacy download pt_core_news_sm
nlp_spacy = spacy.load("pt_core_news_sm")

import unicodedata
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import itertools

from tqdm.notebook import tqdm

import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_colwidth', None) # Remove a largura das colunas do pandas, o que ajuda na visualização no caso do desafio 2.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 25.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
print(os.getcwd())

/content


In [3]:
directory_path = '/content/drive/My Drive/mercado_livre'
os.chdir(directory_path)

In [4]:
os.listdir(directory_path)

['ofertas_relampago.csv',
 'full_devices.csv',
 'items_titles_test.csv',
 'items_titles.csv',
 'Technical Challenge v2.pdf',
 'similar_products_test.csv',
 'Desafio_1.ipynb',
 'Desafio_3.ipynb',
 'Desafio_2.ipynb']

In [5]:
path_items_titles_test = f'{directory_path}/items_titles_test.csv'
path_items_titles = f'{directory_path}/items_titles.csv'

In [6]:
df_items_title_test_orig = pd.read_csv(path_items_titles_test, encoding='utf-8')
df_items_title_orig = pd.read_csv(path_items_titles, encoding='utf-8')

In [7]:
df_items_title_test_orig.head()

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


In [8]:
df_items_title_orig.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Vermelho
1,Tenis Para Caminhada Super Levinho Spider Corrida
2,Tênis Feminino Le Parc Hocks Black/ice Original Envio Já
3,Tênis Olympikus Esportivo Academia Nova Tendência Triunfo
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicleta Carregáve


Verficação de inconsistência na base (nulos)

In [9]:
print('items_title_test_df', df_items_title_test_orig.isnull().sum().sum())
print('items_title_df', df_items_title_orig.isnull().sum().sum())

items_title_test_df 0
items_title_df 0


In [10]:
df_train = df_items_title_orig.copy()
df_test = df_items_title_test_orig.copy()

print(df_train.shape)
print(df_test.shape)

(30000, 1)
(10000, 1)


In [11]:
def remove_accents(text):
  text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
  return text

stop_words = set(stopwords.words('portuguese'))
forbidden_words = r"\b(masculino|feminino)\b"

def preprocess_text(text, forbidden_words=forbidden_words, view=False):

    # Remove accents
    text = remove_accents(text)
    if view:
        print('\nremovendo acentos)\n', text)

    # Lowercase
    text = text.lower()
    if view:
        print('\nmínisculas)\n', text)

    # Remove special characters and numbers (more aggressive cleaning)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    if view:
        print('\nremovendo números e especiais)\n', text)

    # Remove classes gramatícais
    doc = nlp_spacy(text)
    sem_adjetivos = [token.text for token in doc if (token.pos_ != "ADJ") and (token.pos_ != "VERB")]
    if view:
        print('\nanálise morfologica)\n', [(token.text, token.pos_) for token in doc])
    text = " ".join(sem_adjetivos)

    # Remove algumas palavras manualmente
    text = re.sub(forbidden_words, "", text, flags=re.IGNORECASE).strip()

    # Tokenize
    tokens = nltk.word_tokenize(text)
    if view:
        print('\ntokenização)\n', tokens)

    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    if view:
        print('\nmaiores de 2 char)\n', tokens)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    if view:
        print('\nlemmating)\n', tokens)

    return " ".join(tokens)

preprocess_text(df_train['ITE_ITEM_TITLE'].iloc[2], view=True)


removendo acentos)
 Tenis Feminino Le Parc Hocks Black/ice Original Envio Ja

mínisculas)
 tenis feminino le parc hocks black/ice original envio ja

removendo números e especiais)
 tenis feminino le parc hocks blackice original envio ja

análise morfologica)
 [('tenis', 'NOUN'), ('feminino', 'ADJ'), ('le', 'PROPN'), ('parc', 'PROPN'), ('hocks', 'VERB'), ('blackice', 'ADJ'), ('original', 'ADJ'), ('envio', 'NOUN'), ('ja', 'PROPN')]

tokenização)
 ['tenis', 'le', 'parc', 'envio', 'ja']

maiores de 2 char)
 ['tenis', 'parc', 'envio']

lemmating)
 ['tenis', 'parc', 'envio']


'tenis parc envio'

In [12]:
df_train['ITE_ITEM_TITLE*'] = df_train['ITE_ITEM_TITLE'].apply(preprocess_text)
df_test['ITE_ITEM_TITLE*'] = df_test['ITE_ITEM_TITLE'].apply(preprocess_text)

In [13]:
df_train.head()

,ITE_ITEM_TITLE,ITE_ITEM_TITLE*
0,Tênis Ascension Posh Masculino - Preto E Vermelho,tenis ascension
1,Tenis Para Caminhada Super Levinho Spider Corrida,tenis caminhada corrida
2,Tênis Feminino Le Parc Hocks Black/ice Original Envio Já,tenis parc envio
3,Tênis Olympikus Esportivo Academia Nova Tendência Triunfo,tenis academia tendencia
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicleta Carregáve,led bicicleta usb bicicleta


In [14]:
df_test.head()

,ITE_ITEM_TITLE,ITE_ITEM_TITLE*
0,Tênis Olympikus Esporte Valente - Masculino Kids,tenis esporte valente kid
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento,bicicleta barra samy marchas cubo rolamento
2,Tênis Usthemp Slip-on Temático - Labrador 2,tenis slipon labrador
3,Tênis Casual Feminino Moleca Tecido Tie Dye,tenis moleca tie dye
4,Tênis Star Baby Sapatinho Conforto + Brinde,tenis sapatinho


In [15]:
vectorizer = TfidfVectorizer()

tfidf_train_matrix = vectorizer.fit_transform(df_train['ITE_ITEM_TITLE*'])
tfidf_test_matrix = vectorizer.transform(df_test['ITE_ITEM_TITLE*'])

cosine_sim = cosine_similarity(tfidf_test_matrix, tfidf_train_matrix)

In [16]:
pairs = []
n_test = len(df_test)
n_train = len(df_train)
for i in tqdm(range(n_test)):
    max_sim_index = np.argmax(cosine_sim[i])  # Índice do produto mais similar
    pairs.append((df_test.iloc[i]['ITE_ITEM_TITLE'], df_train.iloc[max_sim_index]['ITE_ITEM_TITLE'], cosine_sim[i, max_sim_index]))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [17]:
df_similarities = pd.DataFrame(pairs, columns=['Produto Teste', 'Produto Treino', 'Similaridade'])
df_similarities = df_similarities.set_index('Produto Teste').reindex(df_test['ITE_ITEM_TITLE']).reset_index()
df_similarities

,ITE_ITEM_TITLE,Produto Treino,Similaridade
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Olympikus Valente Kids Masculino,0.808627
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento,Bb De Rolamento Selado Para Mountain Bike De Cinza Prateado,0.366108
2,Tênis Usthemp Slip-on Temático - Labrador 2,Tênis Usthemp Slip-on Temático - Lunna Labrador 2,0.766461
3,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Casual Feminino Moleca Tie Dye 5696203 Confortável,1.000000
4,Tênis Star Baby Sapatinho Conforto + Brinde,Sapatinho Gymboree Azul E Verde,0.990014
...,...,...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil,Chuteira Futsal Oxn Velox 3 Infantil,1.000000
9996,Sapatenis Casual Masculino Estiloso 24horas Conforto Qualida,Tênis Feminino Florido Lindo Envio 24 Horas,0.662773
9997,Tênis Feminino Infantil Molekinha Tie Dye,Tênis Feminino Casual Branco Sapatenis Tie Dye Estiloso,1.000000
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete P/ Treino Azul,Tênis Unissex Barato Leve Ganhe 1 Colchonete P/ Treinos Azul,0.656850


In [18]:
df_similarities.describe()

,Similaridade
count,10000.000000
mean,0.894379
std,0.159728
min,0.000000
25%,0.801529
50%,1.000000
75%,1.000000
max,1.000000


In [19]:
fig = go.Figure()
fig.add_trace(go.Box(y=df_similarities['Similaridade'], name='Similaridade'))
fig.update_layout(title='Distribuição da Similaridade dos Produtos', yaxis_title='Similaridade')
fig.show()

In [20]:
Q1 = df_similarities['Similaridade'].quantile(0.25)
Q3 = df_similarities['Similaridade'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
print(f"Limite Inferior do Boxplot: {limite_inferior:.4f}")

df_similarities[df_similarities['Similaridade'] <= limite_inferior].shape

Limite Inferior do Boxplot: 0.5038


(159, 3)

In [21]:
df_similarities.to_csv(f'{directory_path}/similar_products_test.csv', index=False)